In [4]:
import pandas as pd

In [5]:
dataset = pd.DataFrame(data={'Color': ['G','Y','R','R','Y'],
                             'Diameter': [3,3,1,1,3],
                             'Label': ['A','A','G','G','L']})

In [6]:
dataset

,Color,Diameter,Label
0,G,3,A
1,Y,3,A
2,R,1,G
3,R,1,G
4,Y,3,L


# Algorithm(steps) :-

Build_Tree()

    --> Find Best split

        - Find current Impurity

        - For each column-value combination:

            * Create a question object containing values for column and val. (To be used later for partitioning)

            * Partition data into left and right branches

            * If either partition size == 0, then skip current interation

            * information_gain = current_impurity - average_impurity of partitioned left and right branches

            * If information_gain > best_gain, then best_gain, best_question = information_gain, question

        - return best_gain, best_question

    --> If gain == 0 then return as Leaf_node(row)
    --> left, right = partition(rows)
    --> left branch = Build_Tree(left)
    --> right branch = Build_Tree(right)
    --> return Decision_Node(best_question, left branch, right branch)

In [ ]:
class Question:
    
    def __init__(self, col, value):
        self.column = col
        self.value = value
    
    def match(self, row):
        val = row[self.column]
        if isinstance(val, int) or isinstance(val, float):
            return val >= self.value
        else:
            return val == self.value


In [ ]:
def class_counts(rows):
    counts = {}
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [ ]:
def gini(rows):
    class_count = class_counts(rows)
    impurity = 1
    for each in class_count.keys():
        impurity -= (class_count[each] / len(rows))**2
    return impurity

In [ ]:
def partition(rows, question):
    left_rows, right_rows = [], []
    for row in rows:
        if question.match(row):
            left_rows.append(row)
        else:
            right_rows.append(row)
    return left_rows, right_rows

In [ ]:
def information_gain(left_rows, right_rows, current_impurity):
    weights = len(left_rows) / (len(left_rows) + len(right_rows))
    return current_impurity - weights * gini(left_rows) - (1-weights) * gini(right_rows)

In [ ]:
def best_split(rows):
    best_gain = 0
    best_question = None
    current_impurity = gini(rows)
    n_features = len(rows[0]) - 1
    
    for col in range(n_features-1):
        values = set([row[col] for row in rows])
        
        for val in values:
            # Creating Question Class/object instead of function because (col and val) values 
            # need to be stored for each column and value combination, which will be used later
            question = Question(col, val)
            
            left_rows, right_rows = partition(rows, question)

            if len(left_rows)==0 or len(right_rows)==0:
                continue

            gain = information_gain(left_rows, right_rows, current_impurity)

            if gain > best_gain:
                best_gain, best_question = gain, question
    
    return best_gain, best_question

In [ ]:
class Leaf(rows):

    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [ ]:
class DecisionNode:
    
    def __init__(self, left_branch, right_branch, question):
        self.left_branch = left_branch
        self.right_branch = right_branch
        self.question = question

In [ ]:
def build_tree(rows):
    gain, question = best_split(rows)

    if gain == 0:
        return Leaf(rows)
    
    left_rows, right_rows = partition(rows, question)

    left_branch = build_tree(left_rows)
    right_branch = build_tree(right_rows)

    return DecisionNode(left_branch, right_branch, question)
